In [1]:
class Card:
    suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
    values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10,
              'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}
    side_front = 'front'
    side_back = 'back'

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.side = Card.side_back

    def reveal(self):
        self.side = Card.side_front
        return self

    def get_value(self):
        return Card.values[self.rank]

    def __repr__(self):
        return f'{self.rank} {self.suit} ({self.get_value()})' if self.side == Card.side_front else 'Card'

In [2]:
import random

class Deck:
    def __init__(self):
        self.cards = self.init_cards()
        random.shuffle(self.cards)

    def init_cards(self):
        cards = []
        for suit in Card.suits:
            for rank in Card.values:
                cards.append(Card(suit, rank))
        return cards

    def pop_card(self):
        return self.cards.pop()

In [3]:
class BalanceExceededException(Exception):
    pass 
        
class Player:
    def __init__(self, name):
        self.cards = []
        self.balance = 100
        self.name = name

    def show_cards(self):
        print(self.name + ' cards: ', end='')
        str_cards = [str(card) for card in self.cards]
        print(', '.join(str_cards))

    def pop_cards(self):
        self.cards = []

    def reveal_cards(self):
        for card in self.cards:
            card.reveal()
        self.show_cards()

    def show_balance(self):
        print(self.name + ' balance ' + str(self.balance))

In [4]:
class Game:
    def __init__(self):
        self.deck = None
        self.dealer = Player('Dealer')
        self.player = Player('Player')
        self.current_bet = 0

    def deal(self, player, num_cards, num_revealed=0):
        for _ in range(num_cards):
            player.cards.append(self.deck.pop_card())
        for _ in range(num_revealed):
            player.cards.append(self.deck.pop_card().reveal())
        player.show_cards()

    def hit(self, player, confirm=True):
        if confirm:
            if input('Do you want to hit ? (y/n): ') == 'y':
                self.deal(player, 0, 1)
                if self.calculate_cards(player) < 19:
                    self.hit(player)
        else:
            print(player.name + ' hit')
            self.deal(player, 0, 1)

    def calculate_cards(self, player):
        total = 0
        for card in player.cards:
            total += card.get_value()
        return total

    def make_bet(self):
        while True:
            try:
                bet = int(input('Make your bet: '))
                if bet > self.player.balance and bet > self.dealer.balance:
                    raise BalanceExceededException('One of the players balance exceeded')
            except:
                print('Incorrect value or balance exceeded!')
            else:
                self.current_bet += bet
                print('Your bet is ' + str(bet))
                break

    def adjust_bet(self, player1, player2):
        player1.balance -= self.current_bet
        player2.balance += self.current_bet
        self.current_bet = 0
        player1.show_balance()
        player2.show_balance()

    def start(self):
        print('Deck is ready, the game is started !')
        while True:
            self.deck = Deck()
            self.make_bet()
            self.dealer.show_balance()
            self.deal(self.dealer, 1, 1)
            self.player.show_balance()
            self.deal(self.player, 0, 2)

            while True:
                if self.calculate_cards(self.player) <= 19:
                    self.hit(self.player)
                if self.calculate_cards(self.player) > 21:
                    print('You Burst ! You lost your bet!')
                    self.adjust_bet(self.player, self.dealer)
                    break
                self.dealer.reveal_cards()
                while self.calculate_cards(self.dealer) <= 17 and len(self.deck.cards) > 0:
                    self.hit(self.dealer, False)
                if self.calculate_cards(self.dealer) > 21:
                    print('Dealer Burst ! Dealer lost his bet!')
                    self.adjust_bet(self.dealer, self.player)
                    break

                dealer_total = self.calculate_cards(self.dealer)
                player_total = self.calculate_cards(self.player)
                if dealer_total > player_total:
                    print(f'Dealer {dealer_total} > Player {player_total}. Dealer won !')
                    self.adjust_bet(self.player, self.dealer)
                else:
                    print(f'Dealer {dealer_total} < Player {player_total}. Player won !')
                    self.adjust_bet(self.dealer, self.player)
                break

            if input('Want to play one more round ? (y/n): ') == 'y':
                self.deck = Deck()
                self.dealer.pop_cards()
                self.player.pop_cards()
            else:
                break
    
            

In [5]:
game = Game()
game.start()

Deck is ready, the game is started !
Make your bet: 40
Your bet is 40
Dealer balance 100
Dealer cards: Card, Five Hearts (5)
Player balance 100
Player cards: Two Hearts (2), Nine Clubs (9)
Do you want to hit ? (y/n): y
Player cards: Two Hearts (2), Nine Clubs (9), Four Hearts (4)
Do you want to hit ? (y/n): n
Dealer cards: Six Clubs (6), Five Hearts (5)
Dealer hit
Dealer cards: Six Clubs (6), Five Hearts (5), Nine Spades (9)
Dealer 20 > Player 15. Dealer won !
Player balance 60
Dealer balance 140
Want to play one more round ? (y/n): y
Make your bet: 50
Your bet is 50
Dealer balance 140
Dealer cards: Card, King Clubs (10)
Player balance 60
Player cards: King Spades (10), Queen Diamonds (10)
Dealer cards: Two Diamonds (2), King Clubs (10)
Dealer hit
Dealer cards: Two Diamonds (2), King Clubs (10), Ten Hearts (10)
Dealer Burst ! Dealer lost his bet!
Dealer balance 90
Player balance 110
Want to play one more round ? (y/n): n
